<h1>CNNs for EPRV on HARPS</h1>
The goal here is to training a CNN using HARPS images to the outputs of the HARPS EPRV extraction pipeline to see it a large of NN can replicated more explicit modeling.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [2]:
print(torch.cuda.is_available())

False


In [3]:
torch.cuda.device_count()

0

In [4]:
cuda = torch.device('cuda')
a = torch.tensor([1., 2.], device=cuda)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import subprocess
subprocess.run(["python", "--version"])

In [5]:
import torchvision

In [6]:
subprocess.run(['python','--version'])

NameError: name 'subprocess' is not defined

<h2>Model definition</h2>

Higher stride at higher layers,

In [7]:
class DownSizeNet(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, leaky_slope=0.2):
        super(DownSizeNet, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 3, stride=1, padding=1, bias=False).double()]
        if normalize:
            layers.append(nn.BatchNorm2d(out_size, 0.8).double())
        layers.append(nn.LeakyReLU(leaky_slope).double())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# class DenseLayer(nn.Module):
#     def __init__(self):
#         super(DenseLayer)
    
class RV_Model(nn.Module):
    def __init__(self):
        super(RV_Model, self).__init__()
        self.down1 = DownSizeNet(3, 64, normalize=False)
        self.down2 = DownSizeNet(64, 128)
        self.down3 = DownSizeNet(128, 256)
        self.down4 = DownSizeNet(256, 512)
        self.down5 = DownSizeNet(512, 512)
        self.down6 = DownSizeNet(512, 512)
        self.final = nn.Sequential(
            nn.Conv2d(512, 1, 3, stride=1, padding=1).double(), nn.Tanh().double()
        )
        
        
    def forward(self, x):
        # Propogate noise through fc layer and reshape to img shape
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        
        return self.final(d6)

In [8]:
from torch.utils.data import Dataset
import glob

In [9]:
from astropy.io import fits

<h2>Dataset importing</h2>
The question here is how is the data organized in the directory and how can it be imported with the target RV. 

Not the OG data but after the data is saved from the pre processing step.

In [12]:
import pickle

In [13]:
import numpy as np
import h5py as h5

In [57]:
def h5_to_array(ds):
    rvs_stack = []
    
    img_stack = np.empty((0,3,526,526))
#     rvs_stack = np.empty((0))
    for visit_name,vist_info in ds['visits'].items():
        for hdu_num in ds['images'][visit_name].keys():
            img   = np.array(ds['images'][visit_name][hdu_num])
            if np.sum(img.shape) != 526*2:
                print(np.sum(img.shape))
            if np.sum(img.shape) == (526*2):
                rvs_stack.append(np.double(vist_info.attrs['ESO DRS CCF RVC']))
                all_flats = np.empty([0,526,526])
                for key in vist_info.attrs.keys():
                    if vist_info.attrs[key] == 'FLAT':
                        temp = np.array(ds['images'][key][hdu_num])[None,...]
                        all_flats = np.append(all_flats, temp,axis=0)
                    if vist_info.attrs[key] == 'THAR_THAR':
                        cali = np.array(ds['images'][key][hdu_num])
                        
                img_stack = np.append(img_stack,np.vstack((img,np.median(all_flats,axis=0),cali))[None,...])
                
    return img_stack, np.array(rvs_stack)

In [58]:
class Multi_H5_Dataset(Dataset):
    def __init__(self, h5_files):
        self.h5_files = h5_files
        self.type = torch.Tensor
        self.img_stack = np.empty((0,3,526,526))
        self.rvs_stack = np.empty((0))
        for file in h5_files:
            ds = h5.File(file,'r')
            img_stack, rvs_stack = h5_to_array(ds)
            self.img_stack = np.append(self.img_stack,img_stack,axis=0)
            self.rvs_stack = np.append(self.rvs_stack,rvs_stack,axis=0)
#         print(self.length,len(self.rvs))

    def __getitem__(self, index):
        
        return {'img': self.type(self.img_stack[index,...]).double(), 'rvs': self.rvs[index]}
    
    
    def __len__(self):
        return len(self.rvs_stack)

In [59]:
files1 = glob.glob('../data/HARPS/tryagain/51Peg-2/*.h5')
# files2 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-2/*.h5')
# files3 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-3/*.h5')
# files4 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-4/*.h5')
# file_list = [*files1,*files2,*files3,*files4]

In [60]:
files1

['../data/HARPS/tryagain/51Peg-2/51Peg_0-526_1020-1546.h5',
 '../data/HARPS/tryagain/51Peg-2/51Peg_0-526_510-1036.h5',
 '../data/HARPS/tryagain/51Peg-2/51Peg_0-526_0-526.h5']

In [61]:
dataset = Multi_H5_Dataset([files1[0]])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [ ]:
len(dataset)

In [14]:
import os.path

<h2>Defining Fitting Process</h2>
including hyperparameters, the loss function, and the optimization algo

In [15]:
lr = 0.001
b1 = 0.9
b2 = 0.999

In [16]:
model = RV_Model()
mse_loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(b1, b2))

In [17]:
# directory = something
batch_size = 2
n_cpu = 1
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

NameError: name 'dataset' is not defined

In [18]:
import time
import datetime

<h2>Training Step</h2>
the working step!

In [19]:
import sys

In [22]:
prev_time = time.time()
n_epochs = 1
# validiter = iter(validloader)
for epoch in range(n_epochs):
    for i, batch in enumerate(dataloader):
        
        optimizer.zero_grad()
        something = model(batch['img'])
        y         = something.view(batch['img' t].shape[0], -1).median(1)
        loss      = mse_loss(y,batch['rvs'].double())
        loss.backward()
        optimizer.step()
    
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f]"
                % (
                    epoch,
                    n_epochs,
                    i,
                    len(dataloader),
                    loss.item()
                )  
        )        

NameError: name 'dataloader' is not defined